worker ids:
46994197,CA,Redondo Beach, Salim, 21
47200615,NY,Ossining, Hee Dong, 21, 5
46545976,NY,Newburgh, Eva-Maria, 21, 3
46092070,NY,New City, Sara, 18, 2

In [ ]:
import pandas as pd
from datetime import datetime

name = {46994197:"Salim", 47200615:"Hee Dong", 46545976: "Eva-Maria", 46092070: "Sara"}
file_name = "/dccstor/srosent2/generative/appen/job_1988758NQLFQA.json"
data = pd.read_json(file_name, lines=True)

In [ ]:

skip_workers = {}
time_workers = {}
num_complete = {}

for worker in name:
    skip_workers[worker] = 0
    num_complete[worker] = 0
    # time_workers[worker] = 0

for index, row in data.iterrows():
    info = row['results']['judgments'][0]
    # print(row['results'])
        
    start = datetime.strptime(info['started_at'][11:-6],"%H:%M:%S")
    end = datetime.strptime(info['created_at'][11:-6],"%H:%M:%S")
    if info['worker_id'] not in time_workers:
        time_workers[info['worker_id']] = end-start
    else:
        time_workers[info['worker_id']] += end-start
    num_complete[info['worker_id']] += 1
    print(info['unit_data']['question_id']  + ": " + info['unit_data']['question'])
    print(name[info['worker_id']] + ": " + str(end-start))
    print("PARAGRAPH:")
    for sentence in info['unit_data']['long_answer']:
        print(sentence)
    if 'minimal_answer' in info['unit_data'] and info['unit_data']['minimal_answer'] is not None:
        print("minimal answer: " + info['unit_data']['minimal_answer'])
    if 'skipthere_is_no_answer' in info['data'] or 'there_is_no_answer' in info['data']:
        skip_workers[info['worker_id']] += 1
        print("SKIP")
    else:
        print("ANSWER: ")
        if 'paragraph_sentences' in info['data']:
            for sentence in info['data']['paragraph_sentences']:
                print(sentence)
        else:
            print("no sentences selected")
            print(info['data'])
        if 'type_your_answer_here_keep_your_answer_as_close_to_the_passage_as_possible_' not in info['data']:
        #    print(info['data']['type_your_answer_here_keep_your_answer_as_close_to_the_passage_as_possible_'])
        #else:
            print("no answer provided")
   
    print("---")
    # break
    # print(str(info['worker_id']) + "," + info['region'] + "," + info['city'] + "," + info['started_at'])
print(skip_workers)
for worker in name:
    print(name[worker] + ": " + str(time_workers[worker]/num_complete[worker]) + " skip: " +  str(skip_workers[worker]))

In [1]:
import pandas as pd
from datetime import datetime

name = {47200615:"Hee Dong", 46545976: "Eva Maria", 46092070: "Sara", 45676624: "Mohamed", 46373812: "Chie", 46545946: "Joekie", 46954475: "Arafat", 46994197: "Salim"}
file_name = "/dccstor/srosent2/generative/appen/job_2022794.json"
data = pd.read_json(file_name, lines=True)

In [6]:
from rouge_score import rouge_scorer
scorer = rouge_scorer.RougeScorer(['rougeL'], use_stemmer=True)

def rougeL(answer1, answer2):
    scores = scorer.score(answer1, answer2)
    return scores

def process_data(data):

    avg_rouge_all =  {"Hee Dong": [], "Eva Maria": [], "Sara": [], "Mohamed": [], "Chie": [], "Joekie": [], "Arafat": []} 
    partial =  {"Hee Dong": 0, "Eva Maria": 0, "Sara": 0, "Mohamed": 0, "Chie": 0, "Joekie": 0, "Arafat": 0} 
    
    for index, row in data.iterrows():
        avg_rouge = {"Hee Dong": [], "Eva Maria": [], "Sara": [], "Mohamed": [], "Chie": [], "Joekie": [], "Arafat": []}
        print(str(row['data']['question_id']) + ": " + row['data']['question'])
        print("Title: " + str(row['data']['title']))
        for sentence in row['data']['long_answer']:
            print(sentence)
        print("---")
        judgements = row['results']['judgments']
        skip = set()
        scores = {}

        i = 0
        for i in range(len(judgements)):
            answer_type = judgements[i]['data']['how_would_you_describe_the_questionanswer']

            if answer_type == 'complete' or answer_type == 'partial':
                if answer_type == 'partial':
                    partial[name[judgements[i]['worker_id']]] += 1
                # if 'type_your_answer_here_keep_your_answer_as_close_to_the_passage_as_possible_' not in judgements[i]['data']:
                #     print("Error!")
                #     print(judgements[i])
                #     continue
                # type_your_answer_here_keep_your_answer_as_close_to_the_passage_as_possible_
                answer1 = judgements[i]['data']['type_your_answer_here_it_should_be_concise_and_only_come_from_the_passagetitle_']
                print(name[judgements[i]['worker_id']] + ": " + answer1 + " (" + answer_type + ")")
                if 'comments_optional' in judgements[i]['data']:
                    print("comment: " + judgements[i]['data']['comments_optional'])
            else:
                skip.add(name[judgements[i]['worker_id']] + ": " + answer_type)
                continue

            for j in range(i+1,len(judgements)):
                answer_type = judgements[j]['data']['how_would_you_describe_the_questionanswer']

                if answer_type == 'complete' or answer_type == 'partial':
                    answer2 = judgements[j]['data']['type_your_answer_here_it_should_be_concise_and_only_come_from_the_passagetitle_']
                    # print(str(judgements[j]['worker_id']) + ": " + answer)
                else:
                    continue
                score = rougeL(answer1, answer2)
                if name[judgements[i]['worker_id']] == "Arafat" or name[judgements[i]['worker_id']] == "Sara" \
                     or name[judgements[j]['worker_id']] == "Arafat" or name[judgements[j]['worker_id']] == "Sara":
                    scores[name[judgements[i]['worker_id']] + "-" + name[judgements[j]['worker_id']]] = score['rougeL'].fmeasure
                    avg_rouge[name[judgements[i]['worker_id']]].append(score['rougeL'].fmeasure)
                    avg_rouge[name[judgements[j]['worker_id']]].append(score['rougeL'].fmeasure)
        print("skip: " + str(skip))
        scores = sorted(scores.items(), key=lambda item: item[1], reverse=True)

        for score in scores:
            print(score)

        print("Average: ")
        for worker in avg_rouge:
            if len(avg_rouge[worker]) > 0:
                avg = sum(avg_rouge[worker])/len(avg_rouge[worker])
                print(worker + ": " + str(avg))
                avg_rouge_all[worker].append(avg)
        print("-----------------")

        if index > 20:
            break
    print("Annotator| Avg RougeL|# answered")
    print("-- | -- | --")
    for worker in avg_rouge_all:
        avg = sum(avg_rouge_all[worker])/len(avg_rouge_all[worker])
        print(worker + "|" + str(avg) + "|" + str(len(avg_rouge_all[worker])))
        # print("partial count: " + worker + ": " + str(partial[worker]))

process_data(data)

3920674755999659346: who is in the new england patriots division
Title: AFC East
The American Football Conference -- Eastern Division or AFC East is a division of the National Football League ( NFL ) 's American Football Conference ( AFC ) .
There are currently four teams that reside in the division : the Buffalo Bills ( based in Orchard Park , New York ) ; the Miami Dolphins ( based in Miami Gardens , Florida ) ; the New England Patriots ( based in Foxborough , Massachusetts ) ; and the New York Jets ( based in East Rutherford , New Jersey ) .
---
skip: {'Hee Dong: skip'}
Average: 
-----------------
-4605652307593285005: when was the first tesla model s made
Title: Tesla Model S
The Model S was styled by Franz von Holzhausen , who previously worked for Mazda North American Operations .
The car was codenamed WhiteStar during research and preliminary development .
It was officially announced in a press release on June 30 , 2008 .
The prototype vehicle was displayed at a press conference

ZeroDivisionError: division by zero

This next code snippet is to analyze data when there is one annotation per example

In [22]:

def process_single_data(data):

    annotation_count =  {"Hee Dong": 0, "Eva Maria": 0, "Mohamed": 0, "Chie": 0, "Joekie": 0, "Salim": 0} 
    skip_count =  {"Hee Dong": 0, "Eva Maria": 0, "Mohamed": 0, "Chie": 0, "Joekie": 0, "Salim": 0} 
    sentence_count = {"Hee Dong": 0, "Eva Maria": 0, "Mohamed": 0, "Chie": 0, "Joekie": 0, "Salim": 0} 
    sentences_per_paragraph = {"Hee Dong": 0, "Eva Maria": 0, "Mohamed": 0, "Chie": 0, "Joekie": 0, "Salim": 0} 
    num_with_minimal_answer = {"Hee Dong": 0, "Eva Maria": 0, "Mohamed": 0, "Chie": 0, "Joekie": 0, "Salim": 0} 

    for index, row in data.iterrows():
        judgement = row['results']['judgments'][0]

        # if len(row['data']['long_answer']) < 5:
        #     continue

        if judgement['unit_data']['minimal_text'] != '':
            num_with_minimal_answer[name[judgement['worker_id']]] += 1
            # continue
        
        annotation_count[name[judgement['worker_id']]] += 1

        answer_type = judgement['data']['how_would_you_describe_the_questionanswer']
        sentences_per_paragraph[name[judgement['worker_id']]] += len(row['data']['long_answer'])

        if answer_type == 'complete' or answer_type == 'partial':

            if 'type_your_answer_here_it_should_be_concise_and_only_come_from_the_passagetitle_' not in judgement['data']:
                print(judgement)
                continue
            # if answer_type == 'partial':
            #     partial[name[judgement['worker_id']]] += 1
            if 'paragraph_sentences' in judgement['data']:
                sentence_count[name[judgement['worker_id']]] += len(judgement['data']['paragraph_sentences'])
            answer1 = judgement['data']['type_your_answer_here_it_should_be_concise_and_only_come_from_the_passagetitle_']
            if annotation_count[name[judgement['worker_id']]] < 10:
                print(str(row['data']['question_id']) + ": " + row['data']['question'])
                print("Title: " + str(row['data']['title']))
                
                for sentence in row['data']['long_answer']:
                    if 'paragraph_sentences' in judgement['data'] and sentence in judgement['data']['paragraph_sentences']:
                        print("SELECTED " + sentence)
                    else:
                        print(sentence)
                print(name[judgement['worker_id']] + ": " + answer1 + " (" + answer_type + ")")
                if 'comments_optional' in judgement['data']:
                    print("comment: " + judgement['data']['comments_optional'])
                print("---")
        else:
            skip_count[name[judgement['worker_id']]] += 1
            continue
        
    print("Annotator | # completed | # answered | # skipped | # with minimal answer | # sentences selected |  # sentences per answer | # sentences per paragraph")
    print("-- | -- | -- | -- | -- | -- | -- | -- ")
    for worker in annotation_count:
        print(worker + "|" + str(annotation_count[worker]) +  "|" +
          str(annotation_count[worker] - skip_count[worker]) + "|" + str(skip_count[worker]) + "|" + str(num_with_minimal_answer[worker]) + "|" + str(sentence_count[worker]) + "|" +
          str(round(sentence_count[worker]/(annotation_count[worker]-skip_count[worker]),2)) + "|" +  str(round(sentences_per_paragraph[worker]/(annotation_count[worker]-skip_count[worker]),2)))
        # print("partial count: " + worker + ": " + str(partial[worker]))

process_single_data(data)

-4605652307593285005: when was the first tesla model s made
Title: Tesla Model S
The Model S was styled by Franz von Holzhausen , who previously worked for Mazda North American Operations .
The car was codenamed WhiteStar during research and preliminary development .
It was officially announced in a press release on June 30 , 2008 .
SELECTED The prototype vehicle was displayed at a press conference on March 26 , 2009 .
Exclusive premier of their Model S electric car was held at their Menlo Park store on April 8 , 2009 .
Chie: The prototype vehicle was displayed at a press conference on March 26 , 2009. (complete)
---
-1971174930098368368: how does the gravity on earth compare to the moon
Title: Gravitation of the Moon
SELECTED The acceleration due to gravity on the surface of the Moon is about 1.625 m / s , about 16.6 % that on Earth 's surface or 0.16 ɡ .
Over the entire surface , the variation in gravitational acceleration is about 0.0253 m / s ( 1.6 % of the acceleration due to grav

KeyError: 'paragraph_sentences'